# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
def get_tick_time(data):
    data['df']['sno'] = data['df'].index
    data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
    data['df']['DateTime_frmt']   = [dt.datetime.strptime(x.split(".")[0],"%Y%m%d %H:%M:%S") for x in data["df"]['DateTime']]
    return(data)

In [3]:
def get_ohlc(data):
    
    data['df_bars']['sno'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').sno.first().values.astype(int)
    data['df_bars']['o'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.first().values
    data['df_bars']['h'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.max().values
    data['df_bars']['l'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.min().values
    data['df_bars']['c'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.last().values
    data['df_bars'] = data['df_bars'].reset_index(drop=True)    
    return(data)

In [4]:
def get_sup_res(data):
    if data['fractal_one_side_bar_count'] == 2:
        for i in range(2, len(data['df_bars'])):
            if 1 < i < len(data['df_bars']) - 2:
                if data['df_bars']['l'][i] < data['df_bars']['l'][i-1] < data['df_bars']['l'][i-2] and data['df_bars']['l'][i] < data['df_bars']['l'][i+1] < data['df_bars']['l'][i+2]:
                    data['df_bars']['support'][i] = 'yes'

                if data['df_bars']['h'][i] > data['df_bars']['h'][i-1] > data['df_bars']['h'][i-2] and data['df_bars']['h'][i] > data['df_bars']['h'][i+1] > data['df_bars']['h'][i+2]:
                    data['df_bars']['resistance'][i] = 'yes'

    if data['fractal_one_side_bar_count'] == 1:
        for i in range(1, len(data['df_bars'])):
            if 1 < i < len(data['df_bars']) - 1:
                if data['df_bars']['l'][i] < data['df_bars']['l'][i-1] and data['df_bars']['l'][i] < data['df_bars']['l'][i+1]:
                    data['df_bars']['support'][i] = 'yes'

                if data['df_bars']['h'][i] > data['df_bars']['h'][i-1] and data['df_bars']['h'][i] > data['df_bars']['h'][i+1]:
                    data['df_bars']['resistance'][i] = 'yes'

    return(data)

In [5]:
def combine_nearby_lines(data): 

    data['df'] = pd.merge(data['df'], data['df_bars'], on = 'sno', how = 'left')
    
#     data['sup_res_last_window_df'] = data['df'][-data['sup_res_data_last_ticks_window']:]
#     data['support_values'] = list(data['sup_res_last_window_df']['l'][data['sup_res_last_window_df']['support'] == 'yes'])
#     data['resistance_values'] = list(data['sup_res_last_window_df']['h'][data['sup_res_last_window_df']['resistance'] == 'yes'])

    data['support_values'] = list(data['df']['l'][data['df']['support'] == 'yes'])
    data['resistance_values'] = list(data['df']['h'][data['df']['resistance'] == 'yes'])
    
    data['combined_sup_res'] = list(np.round(data['support_values'] + data['resistance_values'],4))    
    
    check_list = ['No'] * len(data['combined_sup_res'])
    data['dup_removed_sup_res'] = {}
    cntr = 1
    actual_position = 0

    for i in range(len(data['combined_sup_res'])):   
        if check_list[i] == 'No':
            cntr = 1
            check_list[i] =  'Done'

            for j in range(i+1,len(data['combined_sup_res'])):
                if np.abs(data['combined_sup_res'][i] - data['combined_sup_res'][j]) < 0.0002:
                    cntr = cntr + 1
                    check_list[j] = 'Done'
                    #print(i,j, data['combined_sup_res'][i], data['combined_sup_res'][j], np.abs(data['combined_sup_res'][i] - data['combined_sup_res'][j]), cntr, check_list[j])

            actual_position  = actual_position + cntr
            data['dup_removed_sup_res'][data['combined_sup_res'][i]] = cntr

    data['dup_removed_sup_res'] = [k for k in data['dup_removed_sup_res'] if data['dup_removed_sup_res'][k] >= data['sup_res_touches']]        
    
    return(data)

%%time
data['df'] = pd.read_csv('data/EURUSD_2021.csv', nrows=5000)

In [6]:
data['df_bars'] = pd.DataFrame()
data['df_bars']['support'] = ''
data['df_bars']['resistance'] = ''
data['candle_size'] = '5T'
data['fractal_one_side_bar_count'] = 1
data['sup_res_touches'] = 2
data['sup_res_last_ticks_window'] = 10_000

In [7]:
data = get_date_list(data)
data = read_data(data)

Record num : 10000


In [8]:
data = get_tick_time(data)
data = get_ohlc(data)
data = get_sup_res(data)
data = combine_nearby_lines(data)
data = get_rolling_emas(data)

0it [00:00, ?it/s]

Building Lema...


8401it [00:06, 1315.85it/s]
139it [00:00, 1313.72it/s]

Building SLema...


8001it [00:06, 1331.67it/s]
141it [00:00, 1318.50it/s]

Building Sema...


7921it [00:05, 1335.74it/s]


In [9]:
data['df']

,sno,i,DateTime_frmt,Ask,Bid,tick,sema,lema,slema,support,resistance,o,h,l,c
0,2080,0,2021-01-03 23:08:13,1.22377,1.22365,1.223710,1.223715,1.223488,1.223550,NaN,NaN,NaN,NaN,NaN,NaN
1,2081,0,2021-01-03 23:08:14,1.22377,1.22364,1.223705,1.223714,1.223489,1.223551,NaN,NaN,NaN,NaN,NaN,NaN
2,2082,0,2021-01-03 23:08:14,1.22376,1.22364,1.223700,1.223713,1.223489,1.223552,NaN,NaN,NaN,NaN,NaN,NaN
3,2083,0,2021-01-03 23:08:15,1.22374,1.22363,1.223685,1.223712,1.223489,1.223553,NaN,NaN,NaN,NaN,NaN,NaN
4,2084,0,2021-01-03 23:08:16,1.22374,1.22361,1.223675,1.223711,1.223489,1.223554,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,9995,0,2021-01-04 01:07:09,1.22456,1.22450,1.224530,1.224491,1.224198,1.224305,NaN,NaN,NaN,NaN,NaN,NaN
7916,9996,0,2021-01-04 01:07:09,1.22457,1.22451,1.224540,1.224493,1.224198,1.224307,NaN,NaN,NaN,NaN,NaN,NaN
7917,9997,0,2021-01-04 01:07:09,1.22457,1.22452,1.224545,1.224495,1.224199,1.224308,NaN,NaN,NaN,NaN,NaN,NaN
7918,9998,0,2021-01-04 01:07:10,1.22458,1.22452,1.224550,1.224497,1.224199,1.224310,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for i in tqdm(range(0, data['df_len'])):

    data['bid'] = data["df"]['Bid'][i]        
    data['ask'] = data["df"]['Ask'][i]
    data['tick'] = data['df']['tick'][i]        
    data['dt_val'] = data['df']['DateTime_frmt'][i]   
    data['sema'] = data['df']['sema'][i]      
    data['slema'] = data['df']['slema'][i]      
    data['lema'] = data['df']['lema'][i]    

    data = get_position(data)

    # Get Dirs --------------------------------
    if len(data['dir_list']) < 2:
        data['dir_list'].append(data['position'])   
        continue

    elif len(data['dir_list']) == 2:
        data = get_cross_dir(data)
    # ----------------------------------------------------------  

    data = calculate_pl(data)
    data = simple_stop_loss(data)
    data = simple_take_profit(data)   
    data = make_order(data)          

 13%|████▌                              | 1019/7920 [00:00<00:00, 23128.81it/s]


IndexError: list index out of range

In [ ]:
fig = px.line(
    data['df'], 
    x="DateTime_frmt", 
    y=['tick', 'sema','slema', 'lema'], 
    color_discrete_sequence = ['grey', 'red', 'blue', 'black'],
    title='tick chart') 


for val in data['dup_removed_sup_res']:
    fig.add_hline(y = val, line_width=3, line_dash="dot", line_color="darkred")  

fig.show()
